# Demo - Time Series Forecasting with TabPFN  🎉

Welcome to the demo of TabPFN for time series forecasting!

In this demo, we will show you how to use TabPFN for time series forecasting. Concretely, we will:

1. **Load time series data** -- you can also bring in time series data from your own problem
2. **Add features** -- we will use default features, but feel free to experiment with your own features!
3. **Perform prediction** -- zero-shot! 😉
4. **Visualize the forecasting results**


## Setup

## Load Time Series Data

In this demo, we will use the time series dataset from [Chronos Dataset](https://huggingface.co/datasets/autogluon/chronos_datasets) on HuggingFace. Also, we'll keep it short and work with just 2 time series from the dataset.

Feel free to explore other datasets by using the dataset names from Chronos Dataset on HuggingFace or even use time series data of your own problem.

In [1]:
dataset_metadata = {
    "monash_tourism_monthly": {"prediction_length": 24},
    "m4_hourly": {"prediction_length": 48},
}

dataset_choice = "monash_tourism_monthly"
# num_time_series_subset = 16
num_time_series_subset = 150

In [2]:
from datasets import load_dataset
from autogluon.timeseries import TimeSeriesDataFrame

from tabpfn_time_series.data_preparation import to_gluonts_univariate, generate_test_X

prediction_length = dataset_metadata[dataset_choice]['prediction_length']
dataset = load_dataset("autogluon/chronos_datasets", dataset_choice)

tsdf = TimeSeriesDataFrame(to_gluonts_univariate(dataset['train']))
tsdf = tsdf[tsdf.index.get_level_values('item_id').isin(tsdf.item_ids[:num_time_series_subset])]
train_tsdf, test_tsdf_ground_truth = tsdf.train_test_split(prediction_length=prediction_length)
test_tsdf = generate_test_X(train_tsdf, prediction_length)

KeyboardInterrupt: 

Let's take a look at the time series data.

In [ ]:
from tabpfn_time_series.plot import plot_actual_ts

plot_actual_ts(train_tsdf, test_tsdf_ground_truth)

KeyboardInterrupt: 

## Adding Features

In our paper, we propose adding `running_index` and `calendar_features` to the table.

**Feel free to experiment with your own features!**

To do that, simply define your own feature functions and pass them to the `FeatureTransformer`.

In [9]:
from tabpfn_time_series import FeatureTransformer, DefaultFeatures

selected_features = [
    DefaultFeatures.add_running_index,
    DefaultFeatures.add_calendar_features,
]

train_tsdf, test_tsdf = FeatureTransformer.add_features(
    train_tsdf, test_tsdf, selected_features
)


Let's take a look at the tables (train and test) before we proceed to do predictions.

✅ Realize that we have added some features into the tables.

## Prediction

Now, let's perform prediction.

We provide two options, `TabPFNMode.LOCAL` and `TabPFNMode.CLIENT`, as the backend for TabPFN.

- `TabPFNMode.LOCAL` uses your local machine to run TabPFN.
- `TabPFNMode.CLIENT` uses TabPFN's inference service provided by [tabpfn-client](https://github.com/automl/tabpfn-client)

For this demo, we'll use `TabPFNMode.CLIENT` to perform prediction. If you have not use the client before, you'll be prompted to create an account.

Note: if your machine doesn't have a GPU, using `TabPFNMode.CLIENT` is recommended -- must faster 😉.


In [ ]:
from tabpfn_time_series import TabPFNTimeSeriesPredictor, TabPFNMode

predictor = TabPFNTimeSeriesPredictor(
    tabpfn_mode=TabPFNMode.CLIENT,
)

pred = predictor.predict(train_tsdf, test_tsdf)

In [ ]:
import sys
sys.path.append('/home/hoos/hoos-time/playground/tabpfn-time-series-dev/experimental')

from noisy_transform import TabPFNNoisyTranformPredictor

noise_level_candidates = [0.25, 0.5, 1, 2, 10]
all_noisy_preds = {}

for noise_level in noise_level_candidates:
    noisy_predictor = TabPFNNoisyTranformPredictor(
        tabpfn_mode=TabPFNMode.CLIENT,
        noise_level=noise_level,
    )

    noisy_pred = noisy_predictor.predict(train_tsdf, test_tsdf)
    all_noisy_preds[noise_level] = noisy_pred

## Visualize the Results

Let's visualize the forecasting results.

Also, note that we provide both **point prediction** and **quantile prediction**, how amazing! 😄

In [12]:
# from tabpfn_time_series.plot import plot_pred_and_actual_ts

# plot_pred_and_actual_ts(
#     train=train_tsdf,
#     test=test_tsdf_ground_truth,
#     pred=pred,
#     show_quantiles=False,
# )


In [13]:
# plot_pred_and_actual_ts(
#     train=train_tsdf,
#     test=test_tsdf_ground_truth,
#     pred=noisy_pred,
#     show_quantiles=False,
# )

In [ ]:
from autogluon.timeseries.metrics.point import MASE
from autogluon.timeseries.utils.datetime import get_seasonality


MASEComputer = MASE()
MASEComputer.clear_past_metrics()

pred["mean"] = pred["target"]

# Set mean for all noisy predictions
for noise_level, noisy_pred in all_noisy_preds.items():
    all_noisy_preds[noise_level]["mean"] = all_noisy_preds[noise_level]["target"]

MASEComputer.save_past_metrics(
    data_past=train_tsdf,
    seasonal_period=get_seasonality(train_tsdf.freq),
)

clean_mase = MASEComputer.compute_metric(
    data_future=test_tsdf_ground_truth.slice_by_timestep(-prediction_length -1, -1),
    predictions=pred,
)

# Compute MASE for all noise levels
noisy_mase_results = {}
for noise_level, noisy_pred in all_noisy_preds.items():
    noisy_mase_results[noise_level] = MASEComputer.compute_metric(
        data_future=test_tsdf_ground_truth.slice_by_timestep(-prediction_length -1, -1),
        predictions=noisy_pred,
    )

# Return clean MASE and dictionary of noisy MASE results
clean_mase, noisy_mase_results

In [ ]:
pred.head()

# Feedback

Let us know what you think!

Write us at Discord: https://discord.gg/qK7AaXPN or just simply create an issue on [GitHub](https://github.com/liam-sbhoo/tabpfn-time-series/tree/main).

Thank you for trying out our method! 🎉
